In [3]:
import os
import pandas as pd
import pickle
import torch
import io
import urllib
from matplotlib import pyplot as plt
from datasets import load_dataset
from datasets.utils.file_utils import get_datasets_user_agent
import PIL.Image
from torchvision import transforms
from tqdm import tqdm

import utils

USER_AGENT = get_datasets_user_agent()

dir_path = "dataset"
images_dir = "images"
filepath_label = 'filepath'

train_num = 3500 # Expect 15% of these images to not download
valid_num = 900  # Expect 15% of these images to not download

def get_img(url, retries=2):
    for _ in range(retries + 1):
            try:
                request = urllib.request.Request(url, data=None, headers={'user-agent': USER_AGENT})
                with urllib.request.urlopen(request) as req:
                    image = PIL.Image.open(io.BytesIO(req.read()))
                break
            except:
                image = None
    
    return image

def load_data(key, df, num):
    count = 0
    for i in tqdm(range(num)):
        if not df[key][filepath_label][i]:
            pass

        elif df[key][filepath_label][i] == "null":
            url = df[key]['image_url'][i]

            image = get_img(url)
            if not image is None:
                filepath = os.path.join(dir_path, images_dir, f'{key}_{i}.jpg')
                try:
                    image.save(filepath)
                    df[key][filepath_label][i] = filepath
                    count += 1
                except:
                    df[key][filepath_label][i] = None
            else:
                df[key][filepath_label][i] = None
                #print(f'Could not fetch index {i}!')

        else:
            count += 1

    print(f'{count} files retrieved in the "{key}" split.')
    return df

if not os.path.exists(dir_path):
    os.makedirs(dir_path)
    os.makedirs(os.path.join(dir_path, images_dir))

    dset = load_dataset("conceptual_captions")

    filepaths = ["null"] * len(dset['train'])
    dset['train'] = dset['train'].add_column(filepath_label, filepaths)

    filepaths = ["null"] * len(dset['validation'])
    dset['validation'] = dset['validation'].add_column(filepath_label, filepaths)

    df_train = dset['train'].to_pandas()
    df_valid = dset['validation'].to_pandas()

    df = {
        'train': df_train,
        'validation': df_valid
          }

    with open(os.path.join(dir_path, 'df.pkl'), 'wb') as handle:
        pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(os.path.join(dir_path, 'df.pkl'), 'rb') as handle:
    df = pickle.load(handle)

df = load_data('train', df, train_num)
df = load_data('validation', df, valid_num)

with open(os.path.join(dir_path, 'df.pkl'), 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

#image = transforms.Compose([transforms.ToTensor(), utils.Rescale(64)])(image)
#plt.figure(figsize=(8, 8))
#plt.imshow(image.permute(1, 2, 0))
#plt.show()

100%|██████████| 3500/3500 [2:01:51<00:00,  2.09s/it]    


2919 files retrieved in the "train" split.


100%|██████████| 900/900 [53:51<00:00,  3.59s/it]   


710 files retrieved in the "validation" split.


In [5]:
import os
import pandas as pd
import pickle
import torch
import io
import urllib
from matplotlib import pyplot as plt
from datasets import load_dataset
from datasets.utils.file_utils import get_datasets_user_agent
import PIL.Image
from torchvision import transforms, datasets
from tqdm import tqdm

import utils

dir_path = 'aircraft'

datasets.FGVCAircraft(dir_path, download=True)

  0%|          | 0/2753340328 [00:00<?, ?it/s]

Extracting aircraft/fgvc-aircraft-2013b.tar.gz to aircraft


Dataset FGVCAircraft
    Number of datapoints: 6667
    Root location: aircraft

In [2]:
import torch

torch.cuda.is_available()

True

In [3]:
import torch
from transformers import T5Tokenizer, T5EncoderModel

tokenizer = T5Tokenizer.from_pretrained('t5-small')
encoder = T5EncoderModel.from_pretrained('t5-small')


usage: ipykernel_launcher.py [-h]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9008 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"4dc3873a-42f8-4964-aede-eddabd0fe598" --shell=9007 --transport="tcp" --iopub=9009 --f=/afs/csail.mit.edu/u/c/cmarnold/.local/share/jupyter/runtime/kernel-v2-24054TKhKeSZou3vc.json


SystemExit: 2

/storage/cmarnold/miniconda/envs/ml/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
